# Analyzing Tech Employees' Donations To The Clinton and Trump Campaigns

Please see the [main page](https://github.com/BuzzFeedNews/2016-10-tech-employee-contributions) for details.

## Load the data

In [1]:
import pandas as pd
import glob
import re

### Load contributions

In [2]:
MAIN_COLUMNS = [
    "filer_committee_id_number", "form_type",  "transaction_id", "contribution_date",
    "contributor_last_name", "contributor_first_name", "contributor_middle_name",
    "contributor_city", "contributor_state", "contributor_employer", 
    "contribution_amount", "contribution_aggregate",
    "contribution_purpose_descrip", "memo_code", "memo_text_description"
]

In [3]:
def read_individual_contributions(path):
    df = pd.read_csv(path, low_memory=False)
    df["contributor_employer"] = df["contributor_employer"].fillna("").str.strip().str.upper()
    return df[df["entity_type"] == "IND"][MAIN_COLUMNS]

In [4]:
sa17a_files = glob.glob("../data/contributions/*/*-SA17A-*.csv")
sa18_files = glob.glob("../data/contributions/*/*-SA18-*.csv")

In [5]:
contributions = pd.concat([ read_individual_contributions(path)
    for path in (sa17a_files + sa18_files) ])

In [6]:
# Assign committee names to committee IDs
contributions["committee"] = contributions["filer_committee_id_number"].apply({
    "C00580100": "Donald J. Trump For President",
    "C00575795": "Hillary For America"
}.get)

In [7]:
contributions.head().T

,0,1,2,3,4
filer_committee_id_number,C00575795,C00575795,C00575795,C00575795,C00575795
form_type,SA17A,SA17A,SA17A,SA17A,SA17A
transaction_id,C5,C2,C947,C183810,C135530
contribution_date,2.01504e+07,2.01504e+07,2.01504e+07,2.01506e+07,2.01505e+07
contributor_last_name,McAuliffe,McAuliffe,Gensler,Macchio,Benson
contributor_first_name,Terence,Dorothy,Gary,Steven,Barbara
contributor_middle_name,R.,S.,NaN,NaN,NaN
contributor_city,Mc Lean,Mc Lean,Brooklandville,Dix Hills,Cordillera
contributor_state,VA,VA,MD,NY,CO
contributor_employer,COMMONWEALTH OF VIRGINIA,,HILLARY FOR AMERICA,CLEARFLO TECHNOLOGIES,


Millions of dollars received from direct, *itemized* individual contributions (SA17A) and transfers from other authorized committees (SA18):

In [8]:
(contributions.groupby([
    "form_type", "committee"
])["contribution_amount"].sum().unstack().fillna(0) / 1e6).round(1)

,Donald J. Trump For President,Hillary For America
form_type,,
SA17A,33.2,249.7
SA18,42.7,103.8


## Identify contributions from major technology companies

The code below identifies alternative spellings and aliases for 20 major technology companies. For each company, we include "matching" patterns and anti-matching patterns (i.e., similar company names to exclude, e.g., "Amazon Consulting").

In [9]:
employers = [
    ("Adobe", re.compile(r"^ADOBE$|ADOBE, INC|ADOBE SYSTEMS"), None),
    ("Airbnb", re.compile(r"AIRBNB"), re.compile(r"UBER")), # Ignore Uber
    ("Amazon", re.compile(r"AMAZON\b"), re.compile(r"AMAZON CONSULTING|AMAZON PRODUCE|AMAZON INDUSTRIES")), # Ignore Amazon Consulting/Produce/etc.
    ("Apple", re.compile(r"^APPLE$|^APPLE,? INC|\bAPPLE TECH"), None),
    ("Box", re.compile(r"^BOX, INC|^BOX$"), None),
    ("Cisco", re.compile(r"^CISCO,? INC|^CISCO$|^CISCO SYSTEMS"), None),
    ("Dell", re.compile(r"^DELL,? INC|^DELL$|^DELL COMPUTER|^DELL SOFTWARE|^DELL SERVICES"), None),
    ("Ebay", re.compile(r"^EBAY\b"), None),
    ("Facebook", re.compile(r"^FACEBOOK"), None),
    ("Google/Alphabet", re.compile(r"\bGOOGLE|ALPHABET"), re.compile(r"ALPHABET ENERGY|HCL AMERICA|BY PPC")), # Ignore Alphabet Energy, HCL America, PPC
    ("Hewlett-Packard", re.compile(r"HEWLETT.PACKARD|\bHP,? INC|\bHP ENTERPRISE|\bHP LABS"), re.compile(r"EXPERIS")), # Ignore Experis
    ("IBM", re.compile(r"^IBM|\bIBM CO"), None),
    ("Intel", re.compile(r"^INTEL CORP|^INTEL,? INC|^INTEL$"), None),
    ("Microsoft", re.compile(r"MICROSOFT"), None),
    ("Netflix", re.compile(r"NETFLIX"), None),
    ("Oracle", re.compile(r"\bORACLE\b"), re.compile(r"ORACLE LENS")),
    ("Salesforce", re.compile(r"SALESFORCE"), None),
    ("Square", re.compile(r"^SQUARE$|^SQUARE,? INC"), None),
    ("Twitter", re.compile(r"\bTWITTER\b"), None),
    ("Uber", re.compile(r"\bUBER\b"), re.compile(r"DRIVER|PARTNER|AIRBNB|LYFT|NURSECORE|INDEPENDENT|4 SEASONS")), # Ignore seeming contractors
]

In [10]:
contributions["meta_employer"] = None
for name, pat, anti_pat in employers:
    contributions.loc[
        (contributions["contributor_employer"].apply(lambda x: re.search(pat, x) != None)) &
        (contributions["contributor_employer"].apply(lambda x: re.search(anti_pat, x) == None) if anti_pat else True) &
        ~(contributions["contributor_employer"].str.contains("CONTRACTOR"))
    , "meta_employer"] = name

In [11]:
tech_contributions = contributions[
    contributions["meta_employer"].notnull()
]
tech_contributions.head().T

,48,72,80,96,200
filer_committee_id_number,C00575795,C00575795,C00575795,C00575795,C00575795
form_type,SA17A,SA17A,SA17A,SA17A,SA17A
transaction_id,C181210,C47640,C197930,C178000,C58630
contribution_date,2.01505e+07,2.01504e+07,2.01506e+07,2.01505e+07,2.01504e+07
contributor_last_name,Burns,Thomas,Windsheimer,Martin,Fuller
contributor_first_name,Bonnie,Kristin,Marci,Brendon,Jacquelline
contributor_middle_name,NaN,NaN,NaN,NaN,NaN
contributor_city,Rincon,Tega Cay,San Francisco,Villa Park,Orinda
contributor_state,GA,SC,CA,IL,CA
contributor_employer,IBM,MICROSOFT,APPLE INC.,GOOGLE,GOOGLE


## Group contributions by employer

- `itemized_contributions` counts only the itemized contributions listing the employer
- `sum_max_agg` tries to infer the total (including non-itemized contributions), based on the maximum `contribution_aggregate` field for each first-name/last-name/employer combination. This approach isn't foolproof but should, generally, provide an upper-bound estimate for contributions.

In [12]:
tech_contributions_by_employer = pd.DataFrame({
    "itemized_contributions": tech_contributions\
        .groupby([ "meta_employer", "committee" ])["contribution_amount"].sum(),
    "sum_max_agg": tech_contributions.groupby([
        "meta_employer", "committee",
        "contributor_first_name", "contributor_last_name"
    ])["contribution_aggregate"].max().reset_index()\
        .groupby([ "meta_employer", "committee" ])["contribution_aggregate"].sum()
})
tech_contributions_by_employer

itemized_contributions  \
meta_employer   committee                                               
Adobe           Donald J. Trump For President                 1082.40   
                Hillary For America                          68245.01   
Airbnb          Hillary For America                          42723.21   
Amazon          Donald J. Trump For President                 2401.60   
                Hillary For America                         255615.94   
Apple           Donald J. Trump For President                 3646.05   
                Hillary For America                         501574.97   
Box             Hillary For America                          13635.00   
Cisco           Donald J. Trump For President                21386.84   
                Hillary For America                         135871.38   
Dell            Donald J. Trump For President                11402.47   
                Hillary For America                          55987.41   
Ebay            Hillary For America                          93539.09   
Facebook        Donald J. Trump For President                 3719.04   
                Hillary For America                         351010.38   
Google/Alphabet Donald J. Trump For President                20024.00   
                Hillary For America                        1121061.48   
Hewlett-Packard Donald J. Trump For President                10131.55   
                Hillary For America                         152803.50   
IBM             Donald J. Trump For President                24732.96   
                Hillary For America                         276366.49   
Intel           Donald J. Trump For President                 4408.41   
                Hillary For America                         124969.46   
Microsoft       Donald J. Trump For President                25610.78   
                Hillary For America                         552250.33   
Netflix         Donald J. Trump For President                  250.00   
                Hillary For America                          67898.27   
Oracle          Donald J. Trump For President                11292.54   
                Hillary For America                         161048.43   
Salesforce      Donald J. Trump For President                  780.40   
                Hillary For America                         318729.93   
Square          Hillary For America                          23314.40   
Twitter         Donald J. Trump For President                 3800.00   
                Hillary For America                          86338.81   
Uber            Donald J. Trump For President                 1259.60   
                Hillary For America                          36986.30   

                                               sum_max_agg  
meta_employer   committee                                   
Adobe           Donald J. Trump For President      1082.40  
                Hillary For America               73860.48  
Airbnb          Hillary For America               43987.37  
Amazon          Donald J. Trump For President      2912.97  
                Hillary For America              278727.53  
Apple           Donald J. Trump For President      4819.02  
                Hillary For America              536470.97  
Box             Hillary For America               14030.00  
Cisco           Donald J. Trump For President     23421.36  
                Hillary For America              153297.16  
Dell            Donald J. Trump For President     13397.85  
                Hillary For America               66572.09  
Ebay            Hillary For America               96029.64  
Facebook        Donald J. Trump For President      3641.03  
                Hillary For America              362837.11  
Google/Alphabet Donald J. Trump For President     21202.37  
                Hillary For America             1167017.56  
Hewlett-Packard Donald J. Trump For President     11433.00  
                Hillary For America              170

## Total estimated contributions from the 20 companies, by committee

In [13]:
tech_contributions_by_employer.reset_index()\
    .groupby([ "committee" ]).sum()

,itemized_contributions,sum_max_agg
committee,,
Donald J. Trump For President,145928.64,162768.68
Hillary For America,4439969.79,4768564.27


---

# Rough estimate of companies whose employees have donated the most to each campaign

Note: The numbers below are meant only as rough estimates. They do not, for example, attempt to merge sibling/parent companies or misspellings.

### Take basic steps to normalize employer names

- Remove "LLC", "INC", "INCORPORATED", "CO", "LLP", etc. endings.
- Convert all letters to lowercase.
- Convert all non-letters into underscores.

In [14]:
import namestand

In [15]:
cruft_pat = re.compile(r",? (LLC|INC|INCORPORATED|CORPORATION|COROPRATION|CORPRATION|CORP|CO|LLP)$")

In [16]:
def normalize_name(name):
    stripped = re.sub(cruft_pat, "", name.strip().strip("."))
    return namestand.downscore(stripped).strip("_")

In [17]:
contributions["employer_norm"] = contributions["contributor_employer"].apply(normalize_name)

In [18]:
grp_employer_norm = contributions.groupby([ "committee", "employer_norm" ])
by_employer_norm = pd.DataFrame({
    "amount": grp_employer_norm["contribution_amount"].sum(),
    "spellings": grp_employer_norm["contributor_employer"].apply(lambda x: " • ".join(sorted(x.unique())))
}).sort_values("amount", ascending=False)

### Hillary For America

In [19]:
top_employers_clinton = by_employer_norm.loc["Hillary For America"].head(50)
top_employers_clinton

,amount,spellings
employer_norm,,
,71763838.339991,• - • -- • --- • ----- • --------------------...
self_employed,48686372.559995,SELF EMPLOYED • SELF EMPLOYED • SELF- EMPLOYE...
retired,13971228.140001,RETIRED • RETIRED ` • RETIRED.
information_requested,13712874.950000,INFORMATION REQUESTED
not_employed,2991814.180000,NOT EMPLOYED • NOT EMPLOYED • NOT-EMPLOYED
google,1088767.920000,"GOOGLE • GOOGLE INC • GOOGLE INC. • GOOGLE, IN..."
none,784280.900000,--NONE-- • NONE • NONE.
morgan_morgan,574091.250000,MORGAN & MORGAN
microsoft,550750.330000,MICROSOFT • MICROSOFT CORP. • MICROSOFT CORPOR...


### Donald J. Trump For President

In [20]:
top_employers_trump = by_employer_norm.loc["Donald J. Trump For President"].head(50)
top_employers_trump

,amount,spellings
employer_norm,,
retired,23378013.69,RETIRED • RETIRED.
self_employed,10175872.46,SELF - EMPLOYED • SELF - EMPLOYED • SELF -EM...
information_requested,9007485.82,INFORMATION REQUESTED
homemaker,1304884.20,HOMEMAKER
self,175356.10,SELF • SELF LLC
boch_automotive_group,86936.80,BOCH AUTOMOTIVE GROUP
not_employed,84234.93,NOT EMPLOYED
owner,61314.79,OWNER
alliance_coal,46080.00,ALLIANCE COAL • ALLIANCE COAL LLC • ALLIANCE C...


---

## Save results

In [21]:
decimalize = lambda x: float("{0:.2f}".format(x)) if isinstance(x, float) else x

In [22]:
tech_contributions.groupby([ "meta_employer", "contributor_employer"])["contribution_amount"].sum()\
    .apply(decimalize)\
    .to_csv("../output/raw-employer-names.csv")

In [23]:
tech_contributions.sort_values([
        "meta_employer", "contributor_employer",
        "contributor_last_name", "contributor_first_name", "contributor_middle_name",
        "contribution_date"
    ])\
    .applymap(decimalize)\
    .to_csv("../output/tech-contributions.csv", index=False)

In [24]:
tech_contributions_by_employer\
    .applymap(decimalize)\
    .to_csv("../output/tech-employer-totals.csv")

In [25]:
top_employers_clinton\
    .apply(decimalize)\
    .to_csv("../output/top-employers-clinton.csv")

In [26]:
top_employers_trump\
    .apply(decimalize)\
    .to_csv("../output/top-employers-trump.csv")

---

---

---